In [1]:
cd ../..

/home/cridonoso/Shared/cwi_assignament


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval

In [3]:
corpus = pd.read_parquet('./data/corpus.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
def format_table(row):
    table = []
    for x in row:
        table.append(list(x))
    t = pd.DataFrame(table[1:], columns=table[0])
    return t

corpus['table'] = corpus['table'].apply(lambda x: format_table(x)) 

In [58]:
pd.DataFrame(corpus['table'].iloc[2])

,0,1,2,3,4
0,Year,Award,Nominee,Category,Result
1,2013,DJ Magazine Awards,Dyro,Top 100 DJs,30
2,2014,DJ Magazine Awards,Dyro,Top 100 DJs,27
3,2015,DJ Magazine Awards,Dyro,Top 100 DJs,27
4,2016,DJ Magazine Awards,Dyro,Top 100 DJs,93
